In [8]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint ,  solve_ivp
import datetime
import pandas as pd
from sys import platform

if platform == "darwin":
    folder="/Users/sjha/Documents/Github/new_model_data/"  ## mac
if platform == "win32":
    folder="C:\\Users\\jhash\\Documents\\renders\\opto\\" ## windows

print(folder)


C:\Users\jhash\Documents\renders\opto\


In [5]:
import pandas as df

In [2]:
g_y=1.24e6   ##Hz
omega_0=4.45e15 ##Hz
del_omega=1e8
omega_1=omega_0-del_omega ##Hz
ohm=1e6         ##Hz
g_z0=0
g_z1=(20*ohm)**0.5
delta_0=100*ohm

g_y/=ohm
g_z1/=ohm
g_z0/=ohm
omega_1/=ohm
omega_0/=ohm
del_omega/=ohm

In [3]:
def full_ODE(t,z,g_z0,g_z1,g_y,del_omega,N0):
    q_z,p_z,q_y,p_y,a_0,a_p1,a_m1,a_0_d,a_p1_d,a_m1_d=z
    n_0=a_0_d*a_0
    sigma_m=1j*(a_0_d*(a_p1-a_m1)-a_0*(a_p1_d-a_m1_d))
    N_eff=g_z1*(a_p1*a_p1_d+a_m1*a_m1_d)+g_z0*n_0
    q_y_dot     =p_y
    p_y_dot     =-q_y-g_y*sigma_m*N0
    
    q_z_dot     =p_z
    p_z_dot     =-q_z-2*N_eff*N0

    a_0_dot=-1j*(del_omega+g_z0*q_z)*a_0+g_y*q_y*(a_p1-a_m1)/2
    a_p1_dot=-1j*g_z1*q_z*a_p1-g_y*q_y*a_0/2
    a_m1_dot=-1j*g_z1*q_z*a_m1+g_y*q_y*a_0/2
    
    a_0_d_dot=1j*(del_omega+g_z0*q_z)*a_0_d+g_y*q_y*(a_p1_d-a_m1_d)/2
    a_p1_d_dot=1j*g_z1*q_z*a_p1_d-g_y*q_y*a_0_d/2
    a_m1_d_dot=1j*g_z1*q_z*a_m1_d+g_y*q_y*a_0_d/2
    
    x_list=[q_z_dot,p_z_dot,q_y_dot,p_y_dot,a_0_dot,a_p1_dot,a_m1_dot,a_0_d_dot,a_p1_d_dot,a_m1_d_dot]
    return x_list


    
def full_evolve(x_0,a,tf,param_list,N_step=300,method="DOP853"):
    q_z_0,p_z_0,q_y_0,p_y_0=x_0
    g_z0,g_z1,g_y, del_omega, N0=param_list
    z0=np.array([q_z_0,p_z_0,q_y_0,p_y_0,a[0],a[1],a[-1],a[0].conj(),a[1].conj(),a[-1].conj()])
    t = np.linspace(0,tf,N_step)
    xx=solve_ivp(full_ODE, [0,tf],z0,args=param_list,dense_output=False,rtol = 1e-10, atol = 1e-10, t_eval=t,method=method)
    zz=xx.y
    sol=[]
    sol.extend( (np.real(zz[0]),np.real(zz[1]),np.real(zz[2]),np.real(zz[3])) )
    n_0=zz[4]*zz[7]
    n_p1=zz[5]*zz[8]
    n_m1=zz[6]*zz[9]
    N=n_0+n_p1+n_m1
    sigma_m=1j*(zz[7]*(zz[5]-zz[6])-zz[4]*(zz[8]-zz[9]))
    sol.extend( (1-np.real(n_0),np.real(n_p1),np.real(N),np.real(sigma_m)) )
    return sol,t

def Pc(param_list):
    g_z0,g_z1,g_y,del_omega,N0=param_list
    delta=del_omega-2*N0*g_z1*(g_z0-g_z1)+g_z0**2-g_z1**2
    #Pc=g_y**2*N_0/delta
    Pc=(8*delta*g_y**2*N0)/(1-delta**2)**2
    return Pc


In [4]:
def real_ODE(t,z,g_z1,g_y,del_omega,N0):
    q_z,p_z,q_y,p_y,A0_p,A0_m,A1_p,A1_m,E_opt_0,E_opt_1,E_mech_z,E_mech_y,E_c=z

    sigma_m=2*(A1_p*A0_m-A1_m*(A0_p+1))
    q_y_dot=p_y
    p_y_dot=-q_y-g_y*N0*sigma_m
    
    q_z_dot=p_z
#     p_z_dot=-q_z-g_z1*N0*(A1_p**2+A1_m**2)
    p_z_dot=-q_z+2*g_z1*N0*(2*A0_p+A0_p**2+A0_m**2)


    A1_p_dot=-(del_omega-g_z1*q_z)*A1_m-g_y*q_y*(1+A0_p)
    A1_m_dot=+(del_omega-g_z1*q_z)*A1_p-g_y*q_y*A0_m

    A0_p_dot=g_y*q_y*A1_p/2
    A0_m_dot=g_y*q_y*A1_m/2
    
    E_c_dot=2*N0*(A0_p_dot*(A0_p+1)+A0_m_dot*A0_m)*(-g_z1*q_z)+g_y*q_y*N0*(A1_p_dot*A0_m+A1_p*A0_m_dot-A1_m_dot*(A0_p+1)-A1_m*A0_p_dot)-N0*g_z1*(2*A0_p+A0_p**2+A0_m**2)*q_z_dot+g_y*q_y_dot*N0*(A1_p*A0_m-A1_m*(A0_p+1))
    E_mech_dot=(q_y*q_y_dot+p_y*p_y_dot+q_z*q_z_dot+p_z*p_z_dot)/2
    E_mech_z_dot=(q_z*q_z_dot+p_z*p_z_dot)/2
    E_mech_y_dot=(q_y*q_y_dot+p_y*p_y_dot)/2
    E_opt_0_dot=2*N0*(A0_p_dot*(A0_p+1)+A0_m_dot*A0_m)*omega_0
    E_opt_1_dot=N0*(A1_p_dot*A1_p+A1_m_dot*A1_m)*omega_1
    
    x_list=[q_z_dot,p_z_dot,q_y_dot,p_y_dot,A0_p_dot,A0_m_dot,A1_p_dot,A1_m_dot,E_opt_0_dot,E_opt_1_dot,E_mech_z_dot,E_mech_y_dot,E_c_dot]
    return x_list

    
def real_evolve(z0,tf,param_list,N_step=300,method="DOP853"):
    
    t = np.linspace(0,tf,N_step)
    xx=solve_ivp(real_ODE, [0,tf],z0,args=param_list,dense_output=False,rtol = 1e-8, atol = 1e-8, t_eval=t,method=method)
    zz=xx.y
    sol=[]
    for i in range(np.shape(zz)[0]):
        sol.append(np.real(zz[i]))
    return sol,t



In [ ]:
par_col=["del_omega","g_z1","g_z0","g_y","N0","P","qy_0",]
label_col=["time","Q_z","P_z","Q_y","P_y","d_A0_p","d_A0_m","A1_p","A1_m","Eo_0","Eo_1","Em_z","Em_y","Ec"]
columns=par_col+label_col
# df_full=pd.DataFrame(columns=columns)
df_full=pd.read_csv(folder+file_name+'.csv')
file_name="real_p_qy_0_varying"

g_y=1e-1
tf=2e1
N_step=10000
N_grid=10

if P_c<=1:
    tt="stable"
else:
    tt="chaotic"
start=datetime.datetime.now()
P_list=np.linspace(1/N_step,1,N_grid,endpoint=True)
qy_0_list=np.linspace(1/N_step,1,N_grid,endpoint=True)*10**2

label_list=["$Q_z$","$P_z$","$Q_y$","$P_y$","$\delta A_0^+$","$\delta A_0^-$","$A_1^+$","$A_1^-$","$E_{opt}^{(0)}$","$E_{opt}^{(1)}$","$E_{mech}_z$","$E_{mech}_y$","$E_{c}$"]
for i,qy_0 in enumerate(qy_0_list):
    print(100*i/(N_grid-1),"% done")
    for j,P in enumerate(P_list):
        N0=(P*del_omega)**3/(8*g_y**2)
        param_list=[g_z1,g_y,del_omega,N0]   ##g_z0,g_z1,g_y,ohm, omega_0, omega_1, del_omega, N_eq
        z_0=np.array([0,0,qy_0,0,0,0,0,0, N0*del_omega,0,0, qy_0**2/4,0])
        zz,t=real_evolve(z_0,tf,param_list,N_step)
        df = pd.DataFrame(index=range(N_step),columns=columns)
        df.iloc[:N_step, 0:7]=[del_omega,g_z1,g_z0,g_y,N0,P,qy_0]   # Access columns 0 to 4 (inclusive)
        df.iloc[:N_step,7]=t
        df.iloc[:N_step,8:]=np.array(zz).T
        df_full=pd.concat([df_full,df],ignore_index=True)


time_taken=(datetime.datetime.now()-start).total_seconds()
print("total time taken = ",time_taken)

df_full.to_csv(folder+file_name+'.csv', index=False)


0.0 % done
11.11111111111111 % done
22.22222222222222 % done
33.333333333333336 % done
44.44444444444444 % done
55.55555555555556 % done
66.66666666666667 % done
77.77777777777777 % done


In [97]:
file_name="real_p_qy_0_varying"
df = pd.read_csv(folder+file_name+'.csv')
df


,del_omega,g_z1,g_z0,g_y,N0,P,qy_0,time,Q_z,P_z,...,P_y,d_A0_p,d_A0_m,A1_p,A1_m,Eo_0,Eo_1,Em_z,Em_y,Ec
0,100.0,0.004472,0.0,0.1,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,100.0,0.004472,0.0,0.1,0.0,0.0,0.0,2.222222,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,100.0,0.004472,0.0,0.1,0.0,0.0,0.0,4.444444,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,100.0,0.004472,0.0,0.1,0.0,0.0,0.0,6.666667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,100.0,0.004472,0.0,0.1,0.0,0.0,0.0,8.888889,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,100.0,0.004472,0.0,0.1,12500000.0,1.0,100.0,11.111111,67.172404,-271.741077,...,-13322.659295,-0.118462,-0.366136,0.399099,0.135616,-4.941487e+15,4.941488e+15,1.958884e+04,4.438328e+07,6.664430e+07
156,100.0,0.004472,0.0,0.1,12500000.0,1.0,100.0,13.333333,-7138.711508,-1993.337307,...,-5163.115422,-1.995620,0.063473,-0.038727,-0.089012,-2.620733e+14,2.620745e+14,1.373365e+07,6.667516e+06,-1.450935e+07
157,100.0,0.004472,0.0,0.1,12500000.0,1.0,100.0,15.555556,-2073.093389,4214.147948,...,2267.432280,-0.531136,0.856928,-0.302252,-0.018041,-2.549895e+15,2.549896e+15,5.514190e+06,1.293251e+06,5.049610e+07
158,100.0,0.004472,0.0,0.1,12500000.0,1.0,100.0,17.777778,-3256.273363,-4565.663403,...,11430.762819,-1.612428,-0.773802,0.053098,0.222495,-1.455239e+15,1.455240e+15,7.862150e+06,3.266575e+07,-7.823380e+06
